In [1]:
import pandas as pd
import numpy as np
import pathlib
import datetime
from datetime import datetime, timedelta, date
from rich import print
from collections import namedtuple
from dataclasses import dataclass, asdict
import operator
import tomllib
from csvb.ingest import apply_rule, sel_factory, ApplyOp, SelectOp, AOP, SOP, Rule, to_transactions, strip_currency
#from decimal import Decimal, getcontext

#getcontext().prec = 2
pd.options.mode.copy_on_write = True

/var/folders/42/qqsltqsj6fl30wl4xm41gs4m0000gn/T/ipykernel_16874/3553096474.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
def accounts(trans):
    return set(trans["From"]).union(set(trans["To"]))

In [3]:
cfgpth = pathlib.Path.home() / ".config" / "csvb" / "config.toml"
with open(cfgpth, 'rb') as f:
    config = tomllib.load(f)

In [4]:
config

{'title': 'csvb config',
 'paths': {'path': '~/als/csvbdata/',
  'bank': '~/als/csvbdata/bank',
  'products': '~/als/csvbdata/products/'}}

In [5]:
try:
    del apl
except NameError:
    pass

apl = pd.read_csv(pathlib.Path(config["paths"]["bank"]) / "Jan-2024_Apple_CC.csv",
                    header=0, parse_dates=["Transaction Date", "Clearing Date"])

apl = apl.rename(columns={"Amount (USD)": "Amount"})
apl["Amount"] = -1 * apl["Amount"]
apl["From"] = ""

In [6]:
df = apl


rdicts = [dict(select=dict(op="contains", column="Merchant", b="Banana Republic"),        apply=dict(op="assign", column="From", b="Expenses:Clothing")),
          dict(select=dict(op="contains", column="Merchant", b="Apple Services"),         apply=dict(op="assign", column="From", b="Expenses:Subscription")),
          dict(select=dict(op="contains", column="Merchant", b="Phillips 66"),            apply=dict(op="assign", column="From", b="Expenses:Auto:Gas")),
          dict(select=dict(op="contains", column="Merchant", b="Ridge Wallet"),           apply=dict(op="assign", column="From", b="Expenses:Gear")),
          dict(select=dict(op="contains", column="Merchant", b="Martin's Supermarket"),   apply=dict(op="assign", column="From", b="Expenses:Groceries")),
          dict(select=dict(op="contains", column="Merchant", b="Apple Cash Payment"),     apply=dict(op="assign", column="From", b="Income:Apple Cash")),
          dict(select=dict(op="contains", column="Merchant", b="Hulu"),                   apply=dict(op="assign", column="From", b="Expenses:Entertainment")),
          dict(select=dict(op="contains", column="Merchant", b="Trade Coffee CO"),        apply=dict(op="assign", column="From", b="Expenses:Groceries")),
          dict(select=dict(op="contains", column="Merchant", b="Beacon Med"),             apply=dict(op="assign", column="From", b="Expenses:Medical:Body")),
          dict(select=dict(op="contains", column="Merchant", b="South Bend Clinic"),      apply=dict(op="assign", column="From", b="Expenses:Medical:Body")),
          dict(select=dict(op="contains", column="Merchant", b="WMMH"),                   apply=dict(op="assign", column="From", b="Expenses:Entertainment:Self")),
          dict(select=dict(op="contains", column="Merchant", b="Account Ending In 8785"), apply=dict(op="assign", column="From", b="Assets:1st Source:Checking")),
          dict(select=dict(op="contains", column="Merchant", b="bookshop.org"),           apply=dict(op="assign", column="From", b="Expenses:Study")),
          dict(select=dict(op="contains", column="Merchant", b="Patreon"),                apply=dict(op="assign", column="From", b="Expenses:Entertainment")),
          dict(select=dict(op="contains", column="Merchant", b="Monthly Installments"),   apply=dict(op="assign", column="From", b="Liabilities:Apple:Installments")),
          dict(select=dict(op="contains", column="Category", b="Tolls"),                  apply=dict(op="assign", column="From", b="Expenses:Auto:Tolls")),
          dict(select=dict(op="contains", column="Merchant", b="Kobo"),                   apply=dict(op="assign", column="From", b="Expenses:Entertainment:Reading")),
          dict(select=dict(op="contains", column="Merchant", b="Vtsup.com"),              apply=dict(op="assign", column="From", b="Expenses:Entertainment:Self")),
          dict(select=dict(op="contains", column="Merchant", b="City Of South Bend"),     apply=dict(op="assign", column="From", b="Expenses:Utilities")),
          dict(select=dict(op="contains", column="Merchant", b="Andrasi"),                apply=dict(op="assign", column="From", b="Expenses:Home:Cleaning")),
          dict(select=dict(op="contains", column="Merchant", b="Parlevel Texas"),         apply=dict(op="assign", column="From", b="Expenses:Restaurants")),
          dict(select=dict(op="contains", column="Merchant", b="Gannett Newsprpr"),       apply=dict(op="assign", column="From", b="Expenses:Utilities")),
          dict(select=dict(op="contains", column="Description", b="ABEBOOKS"),            apply=dict(op="assign", column="From", b="Expenses:Study"))]



#rdicts = [(dict(op="contains", column="Merchant", b="Banana Republic"),        dict(op="assign", column="To", b="Expenses:Clothing")),
#          (dict(op="contains", column="Merchant", b="Apple Services"),         dict(op="assign", column="To", b="Expenses:Subscription")),
#          (dict(op="contains", column="Merchant", b="Phillips 66"),            dict(op="assign", column="To", b="Expenses:Auto:Gas")),
#          (dict(op="contains", column="Merchant", b="Ridge Wallet"),           dict(op="assign", column="To", b="Expenses:Gear")),
#          (dict(op="contains", column="Merchant", b="Martin's Supermarket"),   dict(op="assign", column="To", b="Expenses:Groceries")),
#          (dict(op="contains", column="Merchant", b="Apple Cash Payment"),     dict(op="assign", column="To", b="Income:Apple Cash")),
#          (dict(op="contains", column="Merchant", b="Hulu"),                   dict(op="assign", column="To", b="Expenses:Entertainment")),
#          (dict(op="contains", column="Merchant", b="Trade Coffee CO"),        dict(op="assign", column="To", b="Expenses:Groceries")),
#          (dict(op="contains", column="Merchant", b="Beacon Med"),             dict(op="assign", column="To", b="Expenses:Medical:Body")),
#          (dict(op="contains", column="Merchant", b="South Bend Clinic"),      dict(op="assign", column="To", b="Expenses:Medical:Body")),
#          (dict(op="contains", column="Merchant", b="WMMH"),                   dict(op="assign", column="To", b="Expenses:Entertainment:Self")),
#          (dict(op="contains", column="Merchant", b="Account Ending In 8785"), dict(op="assign", column="To", b="Assets:1st Source:Checking")),
#          (dict(op="contains", column="Merchant", b="bookshop.org"),           dict(op="assign", column="To", b="Expenses:Study")),
#          (dict(op="contains", column="Merchant", b="Patreon"),                dict(op="assign", column="To", b="Expenses:Entertainment")),
#          (dict(op="contains", column="Merchant", b="Monthly Installments"),   dict(op="assign", column="To", b="Liabilities:Apple Installments")),
#          (dict(op="contains", column="Category", b="Tolls"),                  dict(op="assign", column="To", b="Expenses:Auto:Tolls")),
#          (dict(op="contains", column="Merchant", b="Kobo"),                   dict(op="assign", column="To", b="Expenses:Entertainment:Reading")),
#          (dict(op="contains", column="Merchant", b="Vtsup.com"),              dict(op="assign", column="To", b="Expenses:Entertainment:Self")),
#          (dict(op="contains", column="Merchant", b="City Of South Bend"),     dict(op="assign", column="To", b="Expenses:Utilities")),
#          (dict(op="contains", column="Merchant", b="Andrasi"),                dict(op="assign", column="To", b="Expenses:Home:Cleaning")),
#          (dict(op="contains", column="Merchant", b="Parlevel Texas"),         dict(op="assign", column="To", b="Expenses:Restaurants")),
#          (dict(op="contains", column="Merchant", b="Gannett Newsprpr"),       dict(op="assign", column="To", b="Expenses:Utilities")),
#          (dict(op="contains", column="Description", b="ABEBOOKS"),            dict(op="assign", column="To", b="Expenses:Study"))]

#r = Rule(select=[dict(op="contains", column="Merchant", b="Banana Republic")], apply=[dict(op="assign", column="To", b="Expenses:Clothing")])

rules = []
for d in rdicts:
    rules.append(Rule(**d))


for r in rules:
    apply_rule(r, apl)

In [7]:
try:
    del fst
except NameError:
    pass

fst = pd.read_csv(pathlib.Path(config["paths"]["bank"]) / "Jan-2024_1st.csv",
                    header=3, parse_dates=["Date"])

fst["Amount"] = np.nansum([fst["Amount Credit"],fst["Amount Debit"]], axis=0)
fst["From"] = ""
fst

,Transaction Number,Date,Description,Memo,Amount Debit,Amount Credit,Balance,Check Number,Fees,Amount,From
0,7,2024-01-31,crossfit south b crossfit s,NaN,-80.00,NaN,6553.36,NaN,NaN,-80.00,
1,8,2024-01-31,ND PAYROLL PAYROLL,NaN,NaN,5335.70,6633.36,NaN,NaN,5335.70,
2,9,2024-01-30,INTEREST PAYMENT,NaN,NaN,0.09,1297.66,NaN,NaN,0.09,
3,10,2024-01-25,crossfit south b crossfit s,NaN,-225.00,NaN,1297.57,NaN,NaN,-225.00,
4,11,2024-01-18,NIPSCO ACCOUNTS BILLPAY,NaN,-85.60,NaN,1522.57,NaN,NaN,-85.60,
5,12,2024-01-17,AEP INDIANA MICH BILL PAY,NaN,-84.27,NaN,1608.17,NaN,NaN,-84.27,
6,13,2024-01-16,crossfit south b crossfit s,NaN,-3.00,NaN,1692.44,NaN,NaN,-3.00,
7,14,2024-01-16,KEITH W DAVIS P2P,NaN,NaN,1000.00,1695.44,NaN,NaN,1000.00,
8,7,2024-01-09,ALLY BANK $TRANSFER,NaN,-600.00,NaN,695.44,NaN,NaN,-600.00,
9,8,2024-01-09,VERIZON WIRELESS PAYMENTS,NaN,-110.57,NaN,1295.44,NaN,NaN,-110.57,


In [8]:
fstrdicts = [dict(select=[dict(op="contains", column="Description", b="crossfit south"),
                          dict(op=operator.ne,column="Amount", b=-225.00)], apply=dict(op="assign", column="From", b="Expenses:Groceries")),
             dict(select=[dict(op="contains", column="Description", b="crossfit south"),
                          dict(op=operator.eq,column="Amount", b=-225.00)], apply=dict(op="assign", column="From", b="Expenses:Groceries")),
             dict(select=dict(op="contains", column="Description", b="ND PAYROLL"), apply=dict(op="assign", column="From", b="Income:Notre Dame:Regular")),
             dict(select=dict(op="contains", column="Description", b="INTEREST PAYMENT"), apply=dict(op="assign", column="From", b="Income:Interest")),
             dict(select=dict(op="contains", column="Description", b="NIPSCO"), apply=dict(op="assign", column="From", b="Expenses:Utilities")),
             dict(select=dict(op="contains", column="Description", b="AEP"), apply=dict(op="assign", column="From", b="Expenses:Utilities")),
             dict(select=dict(op="contains", column="Description", b="ALLY BANK \$TRANSFER"), apply=dict(op="assign", column="From", b="Transfer:Ally:Savings")),
             dict(select=dict(op="contains", column="Description", b="VERIZON"), apply=dict(op="assign", column="From", b="Expenses:Utilities")),
             dict(select=[dict(op="contains", column="Description", b="VENMO PAYMENT"),
                          dict(op=operator.eq, column="Date", b=datetime(2024,1,3))], apply=dict(op="assign", column="From", b="Expenses:Gifts")),
             dict(select=dict(op="contains", column="Description", b="KEITH W DAVIS P2P"), apply=dict(op="assign", column="From", b="Transfer:Ally:Savings")),
             dict(select=dict(op="contains", column="Description", b="DOVENMUEHLE MTG MORTG PYMT"), apply=dict(op="assign", column="From", b="Expenses:Home:Mortgage"))


             
                
            ]
for d in fstrdicts:
    apply_rule(Rule(**d), fst)

fst

<>:9: SyntaxWarning: invalid escape sequence '\$'
<>:9: SyntaxWarning: invalid escape sequence '\$'
/var/folders/42/qqsltqsj6fl30wl4xm41gs4m0000gn/T/ipykernel_16874/1731584099.py:9: SyntaxWarning: invalid escape sequence '\$'
  dict(select=dict(op="contains", column="Description", b="ALLY BANK \$TRANSFER"), apply=dict(op="assign", column="From", b="Transfer:Ally:Savings")),


,Transaction Number,Date,Description,Memo,Amount Debit,Amount Credit,Balance,Check Number,Fees,Amount,From
0,7,2024-01-31,crossfit south b crossfit s,NaN,-80.00,NaN,6553.36,NaN,NaN,-80.00,Expenses:Groceries
1,8,2024-01-31,ND PAYROLL PAYROLL,NaN,NaN,5335.70,6633.36,NaN,NaN,5335.70,Income:Notre Dame:Regular
2,9,2024-01-30,INTEREST PAYMENT,NaN,NaN,0.09,1297.66,NaN,NaN,0.09,Income:Interest
3,10,2024-01-25,crossfit south b crossfit s,NaN,-225.00,NaN,1297.57,NaN,NaN,-225.00,Expenses:Groceries
4,11,2024-01-18,NIPSCO ACCOUNTS BILLPAY,NaN,-85.60,NaN,1522.57,NaN,NaN,-85.60,Expenses:Utilities
5,12,2024-01-17,AEP INDIANA MICH BILL PAY,NaN,-84.27,NaN,1608.17,NaN,NaN,-84.27,Expenses:Utilities
6,13,2024-01-16,crossfit south b crossfit s,NaN,-3.00,NaN,1692.44,NaN,NaN,-3.00,Expenses:Groceries
7,14,2024-01-16,KEITH W DAVIS P2P,NaN,NaN,1000.00,1695.44,NaN,NaN,1000.00,Transfer:Ally:Savings
8,7,2024-01-09,ALLY BANK $TRANSFER,NaN,-600.00,NaN,695.44,NaN,NaN,-600.00,Transfer:Ally:Savings
9,8,2024-01-09,VERIZON WIRELESS PAYMENTS,NaN,-110.57,NaN,1295.44,NaN,NaN,-110.57,Expenses:Utilities


In [9]:
try:
    del ally
except NameError:
    pass

ally = pd.read_csv(pathlib.Path(config["paths"]["bank"]) / "Jan-2024_Ally.csv",
                    header=0, parse_dates=["Date"])

ally["From"] = ""
ally = ally.rename(columns=dict(zip(ally.columns, [c.strip() for c in ally.columns])))
ally.columns

Index(['Date', 'Time', 'Amount', 'Type', 'Description', 'From'], dtype='object')

In [10]:
allyrules = [dict(select=dict(op="contains", column="Description", b="transfer"), apply=dict(op="assign", column="From", b="Transfer:Ally:Savings")),
             dict(select=dict(op="contains", column="Description", b="Interest Paid"), apply=dict(op="assign", column="From", b="Income:Interest"))]
for d in allyrules:
    apply_rule(Rule(**d), ally)
ally

,Date,Time,Amount,Type,Description,From
0,2024-02-09,05:41:43,600.00,Deposit,Requested transfer from KEITH W DAVIS Ally Ban...,Transfer:Ally:Savings
1,2024-01-18,23:37:01,36.10,Deposit,Interest Paid,Income:Interest
2,2024-01-12,10:58:32,-1000.00,Withdrawal,Requested transfer to Keith W Davis (1ST SOURC...,Transfer:Ally:Savings
3,2024-01-11,06:12:50,600.00,Deposit,Requested transfer from KEITH W DAVIS Ally Ban...,Transfer:Ally:Savings
4,2023-12-18,00:20:16,32.85,Deposit,Interest Paid,Income:Interest


In [11]:
try:
    del ndfcu
except NameError:
    pass

ndfcu = pd.read_csv(pathlib.Path(config["paths"]["bank"]) / "Jan-2024_NDFCU_CC.csv",
                    header=0, names=["Date", "Note", "Description", "Amount", "Balance"],
                    parse_dates=["Date"])

ndfcu["From"] = ""
ndfcu["Amount"] = ndfcu["Amount"].apply(strip_currency).astype(float)
#ndfcu = ally.rename(columns=dict(zip(ally.columns, [c.strip() for c in ally.columns])))
ndfcu

,Date,Note,Description,Amount,Balance,From
0,2024-01-30,CC SALES DRAFT,AMZN Mktp US*R08 . Amzn.com/bill. WA 0...,-12.03,"$1,556.95",
1,2024-01-30,CC SALES DRAFT,AMZN Mktp US*R02 . Amzn.com/bill. WA 0...,-126.67,"$1,544.92",
2,2024-01-30,CC SALES DRAFT,Netflix.com . netflix.com . CA 0000,-15.49,"$1,418.25",
3,2024-01-30,CC SALES DRAFT,CHEWY.COM . 800-672-4399 . FL 0000,-280.61,"$1,402.76",
4,2024-01-30,CC CREDIT VOUCHER,MED*PREMISE HEAL . 615-577-6605 . TN 0...,25.50,"$1,122.15",
5,2024-01-30,CC CREDIT VOUCHER,MED*PREMISE HEAL . 615-577-6605 . TN 0...,25.50,"$1,147.65",
6,2024-01-28,CC SALES DRAFT,CHIPOTLE 4162 . SOUTH BEND . IN 0000,-15.94,"$1,173.15",
7,2024-01-25,CC SALES DRAFT,LA ESPERANZA . SOUTH BEND . IN 0000,-16.91,"$1,157.21",
8,2024-01-24,CC SALES DRAFT,MED*PREMISE HEAL . BRENTWOOD . TN 0...,-30.00,"$1,140.30",
9,2024-01-21,CC CREDIT VOUCHER,MED*PREMISE HEAL . 615-577-6605 . TN 0...,25.50,"$1,110.30",


In [12]:
ndfcurdicts = [dict(select=dict(op="contains", column="Description", b="AMZN"),        apply=dict(op="assign", column="From", b="Expenses:Home")),
               dict(select=dict(op="contains", column="Description", b="Netflix.com"),        apply=dict(op="assign", column="From", b="Expenses:Subscriptions")),
               dict(select=dict(op="contains", column="Description", b="Amazon Prime"),        apply=dict(op="assign", column="From", b="Expenses:Subscriptions")),
               dict(select=dict(op="contains", column="Description", b="CHEWY"),        apply=dict(op="assign", column="From", b="Expenses:Pet")),
               dict(select=dict(op="contains", column="Description", b="CHIPOTLE"),        apply=dict(op="assign", column="From", b="Expenses:Restaurants")),
               dict(select=dict(op="contains", column="Description", b="LA ESPERANZA"),        apply=dict(op="assign", column="From", b="Expenses:Restaurants")),
               dict(select=dict(op="contains", column="Description", b="CHICORY"),        apply=dict(op="assign", column="From", b="Expenses:Restaurants")),
               dict(select=dict(op="contains", column="Description", b="PREMISE"),        apply=dict(op="assign", column="From", b="Expenses:Medical:Body")),
               dict(select=dict(op="contains", column="Description", b="WALGREENS"),        apply=dict(op="assign", column="From", b="Expenses:Medical:Body")),
               dict(select=dict(op="contains", column="Description", b="NYTimes"),        apply=dict(op="assign", column="From", b="Expenses:Subscriptions")),
               dict(select=dict(op="contains", column="Description", b="COMCAST"),        apply=dict(op="assign", column="From", b="Expenses:Utilities")),
               dict(select=dict(op="contains", column="Description", b="MARTIN S"),        apply=dict(op="assign", column="From", b="Expenses:Groceries")),
               dict(select=dict(op="contains", column="Description", b="FEATHERGILL"),        apply=dict(op="assign", column="From", b="Expenses:Medical:Therapy")),
               dict(select=dict(op="contains", column="Description", b="PHILLIPS 66"),        apply=dict(op="assign", column="From", b="Expenses:Auto:Fuel")),
               dict(select=dict(op="contains", column="Description", b="JESUS"),        apply=dict(op="assign", column="From", b="Expenses:Restaurants")),               
         ]

for d in ndfcurdicts:
    apply_rule(Rule(**d), ndfcu)

ndfcu

,Date,Note,Description,Amount,Balance,From
0,2024-01-30,CC SALES DRAFT,AMZN Mktp US*R08 . Amzn.com/bill. WA 0...,-12.03,"$1,556.95",Expenses:Home
1,2024-01-30,CC SALES DRAFT,AMZN Mktp US*R02 . Amzn.com/bill. WA 0...,-126.67,"$1,544.92",Expenses:Home
2,2024-01-30,CC SALES DRAFT,Netflix.com . netflix.com . CA 0000,-15.49,"$1,418.25",Expenses:Subscriptions
3,2024-01-30,CC SALES DRAFT,CHEWY.COM . 800-672-4399 . FL 0000,-280.61,"$1,402.76",Expenses:Pet
4,2024-01-30,CC CREDIT VOUCHER,MED*PREMISE HEAL . 615-577-6605 . TN 0...,25.50,"$1,122.15",Expenses:Medical:Body
5,2024-01-30,CC CREDIT VOUCHER,MED*PREMISE HEAL . 615-577-6605 . TN 0...,25.50,"$1,147.65",Expenses:Medical:Body
6,2024-01-28,CC SALES DRAFT,CHIPOTLE 4162 . SOUTH BEND . IN 0000,-15.94,"$1,173.15",Expenses:Restaurants
7,2024-01-25,CC SALES DRAFT,LA ESPERANZA . SOUTH BEND . IN 0000,-16.91,"$1,157.21",Expenses:Restaurants
8,2024-01-24,CC SALES DRAFT,MED*PREMISE HEAL . BRENTWOOD . TN 0...,-30.00,"$1,140.30",Expenses:Medical:Body
9,2024-01-21,CC CREDIT VOUCHER,MED*PREMISE HEAL . 615-577-6605 . TN 0...,25.50,"$1,110.30",Expenses:Medical:Body


In [13]:
bank_tables = {"Assets:1st Source:Checking": fst,
               "Liabilities:Apple:Credit Card": apl.rename(columns={"Transaction Date": "Date"}),
              "Assets:Ally:Savings": ally,
              "Liabilities:NDFCU:Credit Card": ndfcu}


trans = to_transactions(bank_tables)
trans

,Date,Description,Amount,From,To
0,2023-12-18,Interest Paid,32.85,Income:Interest,Assets:Ally:Savings
1,2023-12-31,Netflix.com . netflix.com . CA 0000,-15.49,Expenses:Subscriptions,Liabilities:NDFCU:Credit Card
2,2024-01-01,BOOKSHOP.ORG 235 WITHERS STREETUNIT 2 BROOKLYN...,-29.09,Expenses:Study,Liabilities:Apple:Credit Card
3,2024-01-01,PATREON* MEMBERSHIP 600 TOWNSEND 5TH FLOOR INT...,-27.19,Expenses:Entertainment,Liabilities:Apple:Credit Card
4,2024-01-01,AB* ABEBOOKS.CO K7JNI5410 TERRY AVENUE NORTH S...,-48.83,Expenses:Study,Liabilities:Apple:Credit Card
...,...,...,...,...,...
80,2024-01-31,MONTHLY INSTALLMENTS (21 OF 24),-54.12,Liabilities:Apple:Installments,Liabilities:Apple:Credit Card
81,2024-01-31,ACH DEPOSIT INTERNET TRANSFER FROM ACCOUNT END...,2315.13,Assets:1st Source:Checking,Liabilities:Apple:Credit Card
82,2024-01-31,APPLE CASH PAYMENT,40.45,Income:Apple Cash,Liabilities:Apple:Credit Card
83,2024-01-31,crossfit south b crossfit s,-80.00,Expenses:Groceries,Assets:1st Source:Checking


In [14]:
try:
    del bal_decl
except NameError:
    pass

bal_decl = pd.read_csv(pathlib.Path(config["paths"]["bank"]) / "balances.csv",
                  header=0,
                  parse_dates=["Date"])

#bal_decl["From"] = ""
#bal["Amount"] = bal["Amount"].apply(strip_currency).astype(float)
#ndfcu = ally.rename(columns=dict(zip(ally.columns, [c.strip() for c in ally.columns])))
#bal_decl = bal_decl.sort_values("Date")
#bal_decl
#jan_bal = bal_decl.loc[(bal_decl["Date"] >= pd.Timestamp(2023, 12, 31)) &
#                       (bal_decl["Date"] <= pd.Timestamp(2024, 1, 1))
#trans = pd.concat([trans, bal_decl])
bal_decl

,Date,Account,Statement Balance
0,2022-12-31,Liabilities:NDFCU:Credit Card,-76.69
1,2023-11-30,Liabilities:NDFCU:Credit Card,-2085.08
2,2023-10-31,Liabilities:NDFCU:Credit Card,-2492.89
3,2023-09-30,Liabilities:NDFCU:Credit Card,-1581.47
4,2023-08-31,Liabilities:NDFCU:Credit Card,-69.79
5,2023-07-31,Liabilities:NDFCU:Credit Card,-3172.26
6,2023-06-30,Liabilities:NDFCU:Credit Card,-1409.14
7,2023-05-31,Liabilities:NDFCU:Credit Card,-351.48
8,2023-04-30,Liabilities:NDFCU:Credit Card,-3924.01
9,2023-03-31,Liabilities:NDFCU:Credit Card,-2519.52


In [28]:
def create_ledgers(trans):
    """ Create a ledger for each account of all from / to transactions and create balance / time column.
    'Transaction Pair' is a terrible name, but it reminds that this is 1 half of a known transation for use
    as a ledger of an account and running balance calculation.
    All values appended from the "From" side of a transaction are reversed here, because a positive value
    "From" this account subtracts from it's balance.
    
    """
    # TODO: Add an option of a start of / end of period dates. Automatically calculate balance at
    # beginning of period.
    # TODO: Add processing of balance assertions to allow checks and reconciliation.
    
    fl = trans.groupby("From")
    tl = trans.groupby("To")
    ledgers = {}
    for acct in accounts(trans):
        # From accounts, note the negative applied to Amount.
        try:
            df = pd.DataFrame(trans.loc[fl.groups[acct], ["Date", "Description", "To"]]).rename(columns={"To": "Transaction Pair"})
            df["Incoming Amount"] = -1 * trans.loc[fl.groups[acct], "Amount"]
            ledgers[acct] = df
        # If account is not in From group, skip.
        except KeyError:
            pass

        # To accounts.

        try:
            df = pd.DataFrame(trans.loc[tl.groups[acct], ["Date", "Description", "From"]]).rename(columns={"From": "Transaction Pair"})
            df["Incoming Amount"] = trans.loc[tl.groups[acct], "Amount"]
            try:
                ledgers[acct] = pd.concat([ledgers[acct], df])
            # If account was not also in From accounts list, don't concat.
            except KeyError:
                ledgers[acct] = df
        # If account is not in list To accounts, skip.
        except KeyError:
            pass
            
        # Sort and apply cumulative sum.
        ledgers[acct] = ledgers[acct].sort_values("Date")
        ledgers[acct]["Balance"] = ledgers[acct]["Incoming Amount"].cumsum()

    return ledgers

def clean_ledgers(ledgers):
    """Remove transactions occurring before the last transaction with all accounts assigned.
    Intended to catch incomplete data and return a useful ledger set on a per account basis.
    """
    clean = {}
    for acct in ledgers:
        try:
            clean[acct] = ledgers[acct].loc[[ledgers[acct]["From"] == ""]:,:]
        except KeyError:
            clean[acct] = ledgers[acct]
        
    return clean

def init_balance(bal_decl, acct, first_date, leq=False):
    #first_date = min(ledger["Date"])
    if leq:
        acct_balances = bal_decl.loc[(bal_decl["Account"] == acct) & (bal_decl["Date"] <= first_date)]
    else:
        acct_balances = bal_decl.loc[(bal_decl["Account"] == acct) & (bal_decl["Date"] < first_date)]
    
    # Catch indexerror if no balance found.
    try:
        return acct_balances.sort_values("Date").iloc[-1]
    except IndexError:
        return None

    
        

ledgers = create_ledgers(trans)
cl = clean_ledgers(ledgers)
init_balance(bal_decl, 'Assets:Ally:Savings', min(ledgers['Assets:Ally:Savings']["Date"]), leq=True)
ledgers['Assets:1st Source:Checking']

,Date,Description,Transaction Pair,Incoming Amount,Balance
12,2024-01-02,DOVENMUEHLE MTG MORTG PYMT,Expenses:Home:Mortgage,-746.58,-746.58
14,2024-01-02,crossfit south b crossfit s,Expenses:Groceries,-40.00,-786.58
22,2024-01-03,VENMO PAYMENT,Expenses:Gifts,-125.00,-911.58
23,2024-01-04,crossfit south b crossfit s,Expenses:Groceries,-3.00,-914.58
27,2024-01-09,VERIZON WIRELESS PAYMENTS,Expenses:Utilities,-110.57,-1025.15
28,2024-01-09,ALLY BANK $TRANSFER,Transfer:Ally:Savings,-600.00,-1625.15
43,2024-01-16,crossfit south b crossfit s,Expenses:Groceries,-3.00,-1628.15
44,2024-01-16,KEITH W DAVIS P2P,Transfer:Ally:Savings,1000.00,-628.15
45,2024-01-17,AEP INDIANA MICH BILL PAY,Expenses:Utilities,-84.27,-712.42
47,2024-01-18,NIPSCO ACCOUNTS BILLPAY,Expenses:Utilities,-85.60,-798.02


In [32]:
acct = 'Assets:Ally:Savings'
def append_init_row(ledger, acct, bal_decl, leq=False):
    #print(acct)
    #print(ledger)
    # Find proper initial balance.
    init = init_balance(bal_decl, acct, min(ledger["Date"]), leq=leq)

    # Skip if no appropriate balance was found.
    if init is not None:
        
        # Create df of initial balance row.
        decl_to_ledg_col = {"Statement Balance": "Incoming Amount", "Account": "Transaction Pair"}
        init_df = pd.DataFrame(init).T.rename(columns=decl_to_ledg_col)
        init_df["Description"] = "Inital Balance"

        # Add to first row, make sure we are sorted before recalculating balance column.
        ledger = pd.concat([init_df, ledger]).sort_values("Date")
        ledger["Balance"] = ledger["Incoming Amount"].cumsum()
    return ledger

In [30]:
for acct in ledgers:
    ledgers[acct] = append_init_row(ledgers[acct], acct, bal_decl, leq=True)

In [31]:
for acct in ledgers:
    print(acct, ledgers[acct].iloc[0])

Expenses:Utilities Date                                              2024-01-08 00:00:00
Description         GANNETT NEWSPRPR OH 13095 PUBLISHERS DRIVE 888...
Transaction Pair                        Liabilities:Apple:Credit Card
Incoming Amount                                                  9.99
Balance                                                          9.99
Name: 26, dtype: object

Expenses:Gifts Date                       2024-01-03 00:00:00
Description                      VENMO PAYMENT
Transaction Pair    Assets:1st Source:Checking
Incoming Amount                          125.0
Balance                                  125.0
Name: 22, dtype: object

Income:Notre Dame:Regular Date                       2024-01-31 00:00:00
Description                 ND PAYROLL PAYROLL
Transaction Pair    Assets:1st Source:Checking
Incoming Amount                        -5335.7
Balance                                -5335.7
Name: 78, dtype: object

Assets:1st Source:Checking Date                       2023-12-31 00:00:00
Transaction Pair    Assets:1st Source:Checking
Incoming Amount                        2320.59
Description                     Inital Balance
Balance                                2320.59
Name: 25, dtype: object

Expenses:Subscription Date                                              2024-01-01 00:00:00
Description         APPLE.COM/BILL ONE APPLE PARK WAY 866-712-7753...
Transaction Pair                        Liabilities:Apple:Credit Card
Incoming Amount                                                  9.99
Balance                                                          9.99
Name: 5, dtype: object

...

Expenses:Medical:Body Date                                              2024-01-03 00:00:00
Description         BEACON MED GRP ONLINEP100 E. WAYNE ST STE 500 ...
Transaction Pair                        Liabilities:Apple:Credit Card
Incoming Amount                                                 11.05
Balance                                                         11.05
Name: 18, dtype: object

Income:Apple Cash Date                          2024-01-31 00:00:00
Description                    APPLE CASH PAYMENT
Transaction Pair    Liabilities:Apple:Credit Card
Incoming Amount                            -40.45
Balance                                    -40.45
Name: 82, dtype: object

Expenses:Groceries Date                                              2024-01-01 00:00:00
Description         TRADE COFFEE CO 268 SUMMER ST. 6TH FLOOR BOSTO...
Transaction Pair                        Liabilities:Apple:Credit Card
Incoming Amount                                                  35.0
Balance                                                          35.0
Name: 10, dtype: object

Income:Interest Date                2023-12-18 00:00:00
Description               Interest Paid
Transaction Pair    Assets:Ally:Savings
Incoming Amount                  -32.85
Balance                          -32.85
Name: 0, dtype: object

Expenses:Auto:Gas Date                                              2024-01-26 00:00:00
Description         PHILLIPS 66 - 0082 FIRE DAY RD MISHAWAKA 46545...
Transaction Pair                        Liabilities:Apple:Credit Card
Incoming Amount                                                  8.43
Balance                                                          8.43
Name: 65, dtype: object